# MTH 651: Advanced Numerical Analysis
## Lecture 1

### Topics

* Class logistics
  * Lectures
  * Office hours
  * Grading
  * Text book
* Software
  * Canvas
  * Zulip
  * Colab
  * Coeus
* Course goals and topics
* Review of some basic numerical methods

### Coarse Goals

* We are intersted in the **numerical solution** of **partial differential equations**.
* We want to use computer algorithms to obtain _approximate_ (i.e. "numerical") solutions to PDEs
* For the most part, we are interested in PDEs that govern physical systems
  * For example: heat transfer, fluid dynamics, electromagnetic, structural mechanics, etc.
* Obtaining approximate solutions to these PDEs allows us to **simulate** physical systems, in other
  words, we can make predictions about how physical systems will behave without the need to perform
  experiments and in situations where calculations done by hand are impossible

### Warm-Up and Review of Some Basical Numerical Methods

If we want to approximate solutions to differential equations, we should start by approximating
derivatives.

Let $f$ be a given (smooth) function.

Recall the difference quotients:

* Forward difference quotient: $$D_F f(x) = \frac{f(x+h) - f(x)}{h}$$
* Backwards difference quotient: $$D_B f(x) = \frac{f(x) - f(x-h)}{h}$$
* Centered difference quotient: $$D_C f(x) = \frac{f(x+h) - f(x-h)}{2h}$$

Each of these **finite difference quotients** approximates the derivative $f'(x)$.

**Question:** how accurate are these approximations?

<details>
<summary><b>Solution:</b></summary>

Expand $f(x+h)$ and $f(x-h)$ as Taylor series about the point $x$. Find the remainder after
cancelling terms.
</details>

Let's do a simple computational example:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Evaluate our function $f$ and its derivative $f'$ at some sample points.

In [ ]:
def f(x):
   return np.sin(2*np.pi*x)

def df(x):
   return 2*np.pi*np.cos(2*np.pi*x)

n = 51 # number of sample points
x = np.linspace(0, 1, n)
plt.plot(x, f(x), label="$f(x) = \\sin(2 \\pi x)$")
plt.plot(x, df(x), label="$f'(x) = 2 \\pi \\cos(2 \\pi x)$")
plt.legend()
plt.show()

Now let's compute the forward difference quotient.

In [ ]:
D_F = np.zeros(n)
h = x[1]
for i in range(n):
   xi = i*h
   D_F[i] = (f(xi + h) - f(xi)) / h

plt.plot(x, df(x), label="Exact derivative")
plt.plot(x, D_F, label="Forward difference quotient")
plt.legend()
plt.show()

In [ ]:
D_B = np.zeros(n)
h = x[1]
for i in range(n):
   xi = i*h
   D_B[i] = (f(xi) - f(xi - h)) / h

plt.plot(x, df(x), label="Exact derivative")
plt.plot(x, D_B, label="Backward difference quotient")
plt.legend()
plt.show()

In [ ]:
D_C = np.zeros(n)
h = x[1]
for i in range(n):
   xi = i*h
   D_C[i] = (f(xi + h) - f(xi - h)) / (2*h)

plt.plot(x, df(x), label="Exact derivative")
plt.plot(x, D_C, label="Centered difference quotient")
plt.legend()
plt.show()

Let's now try to quantify the error. For simplicitly, we'll estimate $\sin'(1)$, and vary the step
size $h$. Since we know the exact derivative, we can easily compute the error of our approximation.

In [ ]:
# Use these values for h
h = 2.0 ** np.arange(-1, -10, -1)
h

In [ ]:
def f(x):
   return np.sin(x)

def df(x):
   return np.cos(x)

def forward_diff(x, h):
   return (f(x+h) - f(x))/h

def backward_diff(x, h):
   return (f(x) - f(x-h))/h

def centered_diff(x, h):
   return (f(x+h) - f(x-h))/(2*h)

In [ ]:
FD = forward_diff(1, h)
BD = backward_diff(1, h)
CD = centered_diff(1, h)

# Compute the error using the exact answer of 1

FD_error = np.abs(FD - df(1))
BD_error = np.abs(BD - df(1))
CD_error = np.abs(CD - df(1))

In [ ]:
import pandas
pandas.DataFrame(data=np.transpose([FD_error, BD_error, CD_error]), columns=["Forward", "Backward", "Centered"])

It is clear from this table that the centered difference quotient is much more accurate. Let's try
to numerically confirm the rates we expect.

The Taylor series argument showed that the forward and backward difference quotients approximate the
first derivative with an error that scales like $\mathcal{O}(h)$, while the centered difference 
quotient has an error that scales like $\mathcal{O}(h^2)$.

We chose our step size $h$ to be $h_i = 2^{-i}$ for $i = 1, 2, 3, \ldots$

Let $e_F$ denote the error of the forward difference quotient (i.e. $e_F = | D_F f(x) - f'(x) |$).
Then, we know that
$$
   e_F \leq ch + \mathcal{O}(h^2).
$$
So, for $e_{F,i+1}$ (corresponding to step size $h_{i+1}$) we have
$$
\begin{align*}
   e_{F,i+1}
      &\leq ch_{i+1} + \mathcal{O}(h_{i+1}^2) \\
      &= \frac{1}{2} c h_i + \mathcal{O}(h_{i}^2) \\
      &\leq \frac{1}{2} e_{F,i} + \mathcal{O}(h_{i}^2)
\end{align*}
$$
which means that halving $h$ should also approximately halve the error (and the ratio of errors
will approach $\frac{1}{2}$ as $h \to 0$).

On the other hand, the centered difference quotient is _second-order accurate_, meaning
$$
   e_C \leq c h^2 + \mathcal{O}(h^3),
$$
and
$$
\begin{align*}
   e_{C,i+1}
      &\leq ch_{i+1}^2 + \mathcal{O}(h_{i+1}^3) \\
      &= \frac{1}{4} c h_i^2 + \mathcal{O}(h_{i}^3) \\
      &\leq \frac{1}{4} e_{F,i} + \mathcal{O}(h_{i}^3),
\end{align*}
$$
and so halving $h$ should result in a reduction of the error by a factor of 4.

More generally, for $e(h) \sim h^p$ (we would call this a pth-order accurate method), we have $e(rh) \sim r^p h^p \sim r^p e(h)$.

## Emperical rate of convergence

Suppose we have two step sizes, $h_1$ and $h_2$, and we (numerically) compute the errors $e_1$ and
$e_2$. We assume that the method has order of accuracy $p$, i.e. $e \sim h^p$. We will estimate the
**rate** $p$ numerically.

$$
\begin{align*}
   \frac{
      \log\left( e_1 / e_2 \right)
   }{
      \log\left( h_1 / h_2 \right)
   }
      &\sim \frac{
         \log\left( h_1^p/h_2^p \right)
      }{
         \log\left( h_1 / h_2 \right)
      } \\
      &= \frac{
         p \log\left( h_1/h_2 \right)
      }{
         \log\left( h_1 / h_2 \right)
      } \\
      &= p
\end{align*}
$$

In [ ]:
def rate(errors, h_ratio):
   return np.log(errors[:-1]/errors[1:])/np.log(h_ratio)

def prepend_nan(array):
   return np.concatenate(([np.NAN], array))

pandas.DataFrame(
   data=np.transpose([
      FD_error, prepend_nan(rate(FD_error, 2)),
      BD_error, prepend_nan(rate(FD_error, 2)),
      CD_error, prepend_nan(rate(CD_error, 2))
   ]),
   columns=["Forward", "Rate", "Backward", "Rate", "Centered", "Rate"]
)

Another way of seeing the rate (graphically) is to plot the errors on a log-log scale.

The error has the form
$$
e \sim h^p
$$
and so
$$
\log(e) \sim \log(h^p) = p \log(h)
$$
and therefore $\log(e)$ and $\log(h)$ are linearly related with slope $p$.

On a log-log plot, errors of this form will appear linear with slopes corresponding to the rates.

In [ ]:
plt.loglog(1/h, FD_error, label="Forward difference error")
plt.loglog(1/h, BD_error, label="Backward difference error")
plt.loglog(1/h, CD_error, label="Centered difference error")
plt.xlabel("$1/h$")
plt.ylabel("Error")
plt.legend()
plt.show()